#### VAR Automation

#### Description:
###### This file contains the code for VAR automation, along with csv's for intermediate results such as portfolio holdings, price changes for BOTH AAVE AND COMP

###### Author: Nishchal Gaba
###### Created on: January 13, 2022

In [ ]:
# Import packages
import os
import time
import sys
import random
from datetime import datetime, timedelta

import json
import requests
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [1]:
import os
import json
import boto3
import requests
import numpy as np
import pandas as pd
from io import StringIO
from datetime import timedelta
from datetime import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


import matplotlib.pyplot as plt

In [2]:
# Coingecko data fetcher
def getCoinGeckoHistoricalData(_token):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _token(str): Token Symbol (actual token will be fetched from token_names dictionary)
            
        Returns::
            _resDf(pandas dataframe): Returns the dates (yyyy-mm-dd) and prices (USD) in a dataframe
    '''
    try:
        # Making the coingecko historical API request
        coinGeckoTokenCall = token_names[_token]
        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(coinGeckoTokenCall)+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        
        # Converting the result to JSON
        resJSON = request.json()
        
        # Creating empty dataframe
        resDf = pd.DataFrame(resJSON['prices'], columns = ["timestamp", "price"])
        
        # We divide by 1000 to convert the default milliseconds from CoinGecko to seconds
        # Get the human readable date from timestamp
        resDf['timestamp'] = resDf['timestamp'].apply(lambda x: str(datetime.fromtimestamp(x/1000).date()))
        
        # Return the resultant dataframe
        return resDf
        
        
    except Exception as e:
        print("Exception occured while fetching Coingecko Data")
        return str(e)

In [3]:
# AAVE V2 data for a particular date
def aaveV2HistoricalData(_date):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _date(str): date for which data is needed in (%m-%d-%Y) format
            
        Returns::
            _resDf(pandas dataframe): Token, CurrentPrice, availableLiquidity, decimals,
                                        totalLiquidity, utilizationRate, Assets
    '''
    
    try:
        # Starting Index
        ind = 0
        
        # Calling the Aave V2
        aaveV2res = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
        resDf = pd.DataFrame(columns=['Token', 'CurrentPrice', 'availableLiquidity','decimals', 'totalLiquidity','utilizationRate','Assets'])
        
        # Iterating through tokens in AaveV2 API
        for token in aaveV2res:
            try:
                resDf.loc[ind] = [token['symbol']] + [float(token['referenceItem']['priceInUsd'])] + \
                                [float(token['availableLiquidity'])] + [float(token['decimals'])]+ \
                                [float(token['totalLiquidity'])]+ [float(token['referenceItem']['utilizationRate'])] + \
                                [float(token['totalDebt'])]

                ind += 1

            except Exception as e:
                print("**", token, e) #str(asset), str(price_mean), str(price_std), str(cap_mean),str(cap_std), str(vol_mean) , str(vol_std))  
    #             errors.append(token)
                # transactionResponse["error"] = True
                print("Exception occured while fetching Aave V2 Token wise Data")
                return str(e)


        # totalLiquidity == Market
        resDf['totalLiquidity']       = resDf['totalLiquidity'] * resDf['CurrentPrice']
        resDf['Assets']               = resDf['Assets'] * resDf['CurrentPrice']
        resDf['Position']             = resDf['Assets'] - resDf['totalLiquidity']
        resDf['Position($,Billion)']  = resDf['Position'] / pow(10, 9)


        # Return the resultant dataframe
        return resDf
        
    except Exception as e:
        print("Exception occured while fetching Aave V2 API Data")
        return str(e)

In [ ]:
# def change():
#     query = '''
#                 {
#                 reserves
#                 (where:{
#                     symbol:"WBTC"
#                     })
#                 {
#                     id
#                     symbol
#                     name
#                     usageAsCollateralEnabled
#                         borrowingEnabled
#                     baseLTVasCollateral
#                     totalDeposits
#                     totalLiquidity
#                     totalATokenSupply
#                     reserveFactor
                    
#                 }
#                 }
#             '''

#     V2_sample_transport       = RequestsHTTPTransport(
#         url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
#         verify=True,
#         retries=5)
#     V2_client                 = Client(transport=V2_sample_transport)
#     response                  = V2_client.execute(gql(query))

#     # Coingecko Mapping for each token name to fetch historical prices
#     names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         "REN"   : "ren",
#         "RENFIL": "renfil",
#         "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }
    
    
#     columns = []
#     columns.append('timestamp')
#     for key in list(names.keys()):
#         columns.append(key+'_price')
#         columns.append(key+'_%change(10)')


#     df        = pd.DataFrame(columns = columns)
#     LOOK_BACK = 375
#     LIMIT     = 5
#     WBTC      = []

#     request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#     data      = request.json()
#     WBTC      = data['prices']
#     tt        = []
#     P         = []
#     for day in data['prices'][-LOOK_BACK:]:
#         tt.append(day[0]/1000)
#         P.append(day[1])
#     df['WBTC_price'] = P
#     df['timestamp']  = tt

#     track = {}
#     for asset in list(names.keys()):#[:LIMIT]:
#         if asset == 'WBTC':
#             continue
#         request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#         data = request.json()
#         tt = []
#         P = []
        
#         if len(data['prices']) < LOOK_BACK:
#             req = LOOK_BACK - len(data['prices'])
#             addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
#             data['prices'] = addition + data['prices']
#             track[asset] = req
#         else:
#             data['prices'] = data['prices'][-LOOK_BACK:]
        
#         for day in data['prices']:
#             tt.append(day[0]/1000)
#             P.append(day[1])
#         df[asset+'_price'] = P
#         df['timestamp']  = tt

#     df['timestamp'] = tt

#     for asset in list(names.keys()):#[:LIMIT]:
        
#         diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
#         change = []
#         for i in range(10):
#             change.append('Nan')
#         for i in range(10,df.shape[0]):
#             change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#         df[asset+'_%change(10)'] = change
        
#         diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
#         change = []
#         for i in range(10):
#             change.append('Nan')
#         for i in range(10,df.shape[0]):
#             change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#         df[asset+'_%change(1)'] = change
        
#     for key in track.keys():
#         df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

#     df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
#     df  = df.iloc[::-1]
#     df  = df.reset_index()
#     df = df.drop(['index'], axis=1)

        
#     return df

In [ ]:
# # Coingecko Mapping for each token name to fetch historical prices
# names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         "REN"   : "ren",
#         "RENFIL": "renfil",
#         "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }

In [ ]:
# # Funtion to give historical price changes for each token mentioned in Names Dictionary
# def change():
#     columns = []
#     columns.append('timestamp')
#     for key in list(names.keys()):
#         columns.append(key+'_price')
#         columns.append(key+'_%change(10)')
#     df        = pd.DataFrame(columns = columns)
#     LOOK_BACK = 375 * 2
#     #LIMIT     = 5
#     WBTC      = []
#     request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#     data      = request.json()
#     WBTC      = data['prices']
#     tt        = []
#     P         = []
#     for day in data['prices'][-LOOK_BACK:]:
#         tt.append(day[0]/1000)
#         P.append(day[1])
#     df['WBTC_price'] = P
#     df['timestamp']  = tt
#     track = {}
#     for asset in list(names.keys()):#[:LIMIT]:
#         if asset == 'WBTC':
#             continue
#         request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#         P    = []
#         if request.status_code != 200:
#             data = {}
#             data['prices'] = WBTC[ -LOOK_BACK: ]
#             for day in data['prices']:
#                 P.append(day)
#         else:   
#             data = request.json()
#             tt   = []
            

#             if len(data['prices']) < LOOK_BACK:
#                 req = LOOK_BACK - len(data['prices'])
#                 addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
#                 data['prices'] = addition + data['prices']
#                 track[asset] = req
#             else:
#                 data['prices'] = data['prices'][-LOOK_BACK:]

#             for day in data['prices']:
#                 tt.append(day[0]/1000)
#                 P.append(day[1])
                
#         df[asset+'_price'] = P
#         df['timestamp']  = tt
#     df['timestamp'] = tt
    
#     print("PRICES FETECHED")
    
#     unknown_assets = ['PAX']
#     for asset in list(names.keys()):#[:LIMIT]:
#         try:
#             change = []
#             diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
            
#             for i in range(10):
#                 change.append('Nan')
#             for i in range(10,df.shape[0]):
#                 change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#             df[asset+'_%change(10)'] = change

#             diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
#             change = []
#             for i in range(10):
#                 change.append('Nan')
#             for i in range(10,df.shape[0]):
#                 change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#             df[asset+'_%change(1)'] = change  
            
#         except Exception as e:
#             unknown_assets.append(asset)
#             print("Error ", asset, e)
    
#     for asset in unknown_assets:
#         df[asset+'_%change(1)']  = df['WBTC_%change(1)'] 
#         df[asset+'_%change(10)'] = df['WBTC_%change(10)'] 
#     df['PAX_%change(10)'] = df['WBTC_%change(10)'] 
#     df['PAX_%change(1)'] = df['WBTC_%change(1)'] 
        
#     for key in track.keys():
#         df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

#     df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
#     df  = df.iloc[::-1]
#     df  = df.reset_index()
#     df = df.drop(['index'], axis=1)

#     df = df[:-10]

#     print("CHECK POINT ")
#     return df
# change_df = change()

In [4]:
# Coingecko Mapping for each token name to fetch historical prices
names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        "REN"   : "ren",
        "RENFIL": "renfil",
        "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }
# Funtion to give historical price changes for each token mentioned in Names Dictionary
def change():
    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        P    = []
        if request.status_code != 200:
            data = {}
            data['prices'] = WBTC[ -LOOK_BACK: ]
            for day in data['prices']:
                P.append(day)
        else:   
            data = request.json()
            tt   = []
            

            if len(data['prices']) < LOOK_BACK:
                req = LOOK_BACK - len(data['prices'])
                addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
                data['prices'] = addition + data['prices']
                track[asset] = req
            else:
                data['prices'] = data['prices'][-LOOK_BACK:]

            for day in data['prices']:
                tt.append(day[0]/1000)
                P.append(day[1])
                
        df[asset+'_price'] = P
        df['timestamp']  = tt
    df['timestamp'] = tt
    
    print("PRICES FETECHED")
    
    unknown_assets = ['PAX']
    for asset in list(names.keys()):#[:LIMIT]:
        try:
            change = []
            diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
            
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(10)'] = change

            diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
            change = []
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(1)'] = change  
            
        except Exception as e:
            unknown_assets.append(asset)
            print("Error ", asset, e)
    
    for asset in unknown_assets:
        df[asset+'_%change(1)']  = df['WBTC_%change(1)'] 
        df[asset+'_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(1)'] = df['WBTC_%change(1)'] 
        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)

    df = df[:-10]

    print("CHECK POINT ")
    return df
# change_df = change()

In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [ ]:
# query = '''
#                 {
#                 reserves
#                 (where:{
#                     symbol:"WBTC"
#                     })
#                 {
#                     id
#                     symbol
#                     name
#                     usageAsCollateralEnabled
#                         borrowingEnabled
#                     baseLTVasCollateral
#                     totalDeposits
#                     totalLiquidity
#                     totalATokenSupply
#                     reserveFactor
                    
#                 }
#                 }
#             '''

In [ ]:
# V2_sample_transport       = RequestsHTTPTransport(
#         url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
#         verify=True,
#         retries=5)

In [5]:
# Required Date
# By default set to current date
# requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate = '12-16-2021' #"01-10-2022" #mm-dd-yyyy

In [10]:
def VARDate(_date):
    '''
        Gets the VAR results, AaveV2 for a particular position and price change dataframe
        
        Args::
            _date(str): Date in "%m-%d-%Y" format
        
        Returns::
            aaveV2Res(pandas dataframe): AaveV2 Historical Data for the given date
            df (pandas dataframe): Price change dataframe
    '''
    # Aave V2 Data for particular date
    aaveV2Res = aaveV2HistoricalData(_date)
    
    # Price Change Dataframe
    df = change()
    
    # Flipping the date format
    reqDate = datetime.strptime(_date, '%m-%d-%Y').strftime('%Y-%m-%d')
    
    # Converting str to datetime
    df['date']= pd.to_datetime(df['date'])
    # Remove dates above the required date
    mask = (df['date'] <= reqDate)
    df = df.loc[mask]
    # Converting the date column back to str
    df['date']=df['date'].astype(str)
    
    REQUIRED_1 = []
    REQUIRED_10 = []
    historicalVAR = pd.DataFrame(columns=['date', 'VAR_10', 'VAR_1'])
    hist_ind = 0
    
    cols     = aaveV2Res['Token']
    FINAL_DF = aaveV2Res[['Position($,Billion)']].T
    FINAL_DF.columns = cols
    # Sliding Window Var Calculator
    VAR      = []
    VAR_DATE = []
    errors   = []
    for ind, ROW in enumerate(FINAL_DF.iterrows()):
        REQUIRED_10 = []
        for required_row in df[ind:ind+365].iterrows():
            SUM_10  = []
            for key in ROW[1].keys():
                if isinstance(ROW[1][key], list):
                    ROW[1][key] = 0
                SUM_10.append((float(ROW[1][key]) * required_row[1][key+'_%change(10)'])/100)
            REQUIRED_10.append(sum(SUM_10))
        VAR.append(sorted(REQUIRED_10, key=float)[3])
        VAR_DATE.append(df[ind:ind+365].at[REQUIRED_10.index(sorted(REQUIRED_10, key=float)[3]) + ind, 'date'])
    
    testdf = pd.DataFrame(columns=['SYMBOL', 'totalLiquidity', 'totalDebt'])
    ind = 0
    
    AAVE_API_RESULT  = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
    
    for i in AAVE_API_RESULT:
        testdf.loc[ind] = [i['symbol']] + [float(i['totalLiquidity']) * float(i['referenceItem']['priceInUsd']) ]\
                    + [float(i['totalDebt']) * float(i['referenceItem']['priceInUsd'] )] #+ [i['referenceItem']['priceInUsd'] * ]
        ind += 1

    TOTAL_LIABILITIES    = aaveV2Res['totalLiquidity'].sum()
    TOTAL_ASSETS         = aaveV2Res['Assets'].sum()
    VAR_10_DAY_99_P      = VAR
    VAR_DATE_10_DAY_99_P = VAR_DATE

#     RELATIVE_VAR_ASSETS = str(VAR * pow(10,9) / TOTAL_ASSETS)
#     RELATIVE_VAR_LIABILITIES = str(VAR * pow(10,9) / TOTAL_LIABILITIES)
    #TOTAL_ASSETS
    #TOTAL_LIABILITIES
    return aaveV2Res, df, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P

In [11]:
# aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)
# Date in %m-%d-%Y format
aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)

PRICES FETECHED
Error  SUSD unsupported operand type(s) for -: 'list' and 'list'
Error  ENJ unsupported operand type(s) for -: 'list' and 'list'
Error  REN unsupported operand type(s) for -: 'list' and 'list'
Error  SNX unsupported operand type(s) for -: 'list' and 'list'


/Users/rahulkanojia/miniconda3/envs/env/lib/python3.7/site-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CHECK POINT 


In [12]:
VAR_10_DAY_99_P

[-4.450595218045071]

In [13]:
VAR_DATE_10_DAY_99_P

['2021-02-06']

In [ ]:
type(priceChangeDf['date'])

In [ ]:
# Token change in price for AAVE VAR Date
idx = priceChangeDf.index[priceChangeDf['date'] == VAR_DATE_10_DAY_99_P].tolist()
priceChangeVARDATE = priceChangeDf.loc[idx]
# _index = priceChangeDf.index[mask]
# priceChangeDf[_index]

In [ ]:
aaveV2Res = aaveV2HistoricalData(requiredDate)

In [ ]:
# Price Change Dataframe
df = change()

In [ ]:
# Flipping the date format
reqDate = datetime.strptime(requiredDate, '%m-%d-%Y').strftime('%Y-%m-%d')
# Converting str to datetime
df['date']= pd.to_datetime(df['date'])
# Remove dates above the required date
mask = (df['date'] <= reqDate)
df = df.loc[mask]
# Converting the date column back to str
df['date']=df['date'].astype(str)

In [ ]:
for name in aaveV2Res['Token']:
    print(name, ": ", df[:365][name+'_%change(10)'].sum() )

In [ ]:
REQUIRED_1 = []
REQUIRED_10 = []
historicalVAR = pd.DataFrame(columns=['date', 'VAR_10', 'VAR_1'])
hist_ind = 0

In [ ]:
for i in FINAL_DF.iterrows():
    break

In [ ]:
df.shape

In [ ]:
VAR_DATE

In [ ]:


for required_row in df[:365].iterrows():
    required_row = required_row[1]
    SUM_10 = []
    SUM_1 = []

    for row in aaveV2Res.iterrows():
        try:
            SUM_10.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(10)'])/100))
            SUM_1.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(1)'])/100))
        except Exception as e:
            print("ERROR", e)
            pass

    REQUIRED_1.append(sum(SUM_1))
    REQUIRED_10.append(sum(SUM_10))

    historicalVAR.loc[hist_ind] = [required_row['date']] + [sum(SUM_10)] + [sum(SUM_1)]
    hist_ind += 1 

# Historical VAR for past 362 days    
historicalVAR = historicalVAR[:365]



REQUIRED_10 = historicalVAR.sort_values('VAR_10').iloc[3]
REQUIRED_1 = historicalVAR.sort_values('VAR_1').iloc[3]

REQUIRED_10_95 = historicalVAR.sort_values('VAR_10').iloc[9]
REQUIRED_1_95 = historicalVAR.sort_values('VAR_1').iloc[9]

var_date_10_99p = REQUIRED_10['date']
var_date_10_95p = REQUIRED_10_95['date']
var_date_1_99p = REQUIRED_1['date']
var_date_1_95p = REQUIRED_1_95['date']

#     print(historicalVAR)

testdf = pd.DataFrame(columns=['SYMBOL', 'totalLiquidity', 'totalDebt'])
ind = 0

AAVE_API_RESULT  = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()

for i in AAVE_API_RESULT:
    testdf.loc[ind] = [i['symbol']] + [float(i['totalLiquidity']) * float(i['referenceItem']['priceInUsd']) ]\
                + [float(i['totalDebt']) * float(i['referenceItem']['priceInUsd'] )] #+ [i['referenceItem']['priceInUsd'] * ]
    ind += 1

TOTAL_LIABILITIES = aaveV2Res['totalLiquidity'].sum()
TOTAL_ASSETS      = aaveV2Res['Assets'].sum()
VAR_10_DAY_99_P = str(REQUIRED_10['VAR_10'])
VAR_DATE_10_DAY_99_P = var_date_10_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['10_day_95p'] = str(REQUIRED_10_95['VAR_10'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_10_day_95p'] = var_date_10_95p

#     # transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1[5])
#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_99p'] = var_date_1_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_95p'] = str(REQUIRED_1_95['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_95p'] = var_date_1_95p

RELATIVE_VAR_ASSETS = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_ASSETS)
RELATIVE_VAR_LIABILITIES = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_LIABILITIES)
#TOTAL_ASSETS
#TOTAL_LIABILITIES
return aaveV2Res, df, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P

In [ ]:
# Contribution to VAR
# -1 * (TOKEN_PRICE * TOKEN_POSITION * TOKEN%CHANGE)

In [ ]:
# PLATFORM, TOKEN, POSITION_ON_INPUT_DATE, POSITION_in_B_ON_INPUT_DATE, TOKEN_%_CHANGE_ON_VAR_DATE, CONTRIBTION TO VAR

In [ ]:
column_names = ["PLATFORM", "TOKEN", "POSITION_ON_INPUT_DATE", "POSITION_ON_INPUT_DATE_BILLIONS",
               "TOKEN_%_CHANGE_ON_VAR_DATE", "CONTRIBUTION_TO_VAR"]
resultDf = pd.DataFrame(columns = column_names)

In [ ]:
# Itearing through the position in Aave

# for token in aaveV2Res["Token"]:
for idx, row in aaveV2Res.iterrows():
    try:
        temp = []
        
        temp.append("AAVE")
        temp.append(str(aaveV2Res["Token"][idx]))
        temp.append(aaveV2Res["Position"][idx])
        temp.append(aaveV2Res["Position($,Billion)"][idx])
        temp.append(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])
        temp.append(float(-1 * float(aaveV2Res["Position($,Billion)"][idx]) * float(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])))
        
        
        
        # Add the values to the latest row
        df_length = len(resultDf)
        resultDf.loc[df_length] = temp
        
    except:
        print("Exception occured for the token: "+str(token))

# print(temp)

In [ ]:
# Sample Final Res
resultDf

### Compound Code

In [ ]:
def change():

    names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        # "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        # "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        # "REN"   : "ren",
        "RENFIL": "renfil",
        # "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }
    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data = request.json()
        tt = []
        P = []
        
        if len(data['prices']) < LOOK_BACK:
            req = LOOK_BACK - len(data['prices'])
            addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
            data['prices'] = addition + data['prices']
            track[asset] = req
        else:
            data['prices'] = data['prices'][-LOOK_BACK:]
        
        for day in data['prices']:
            tt.append(day[0]/1000)
            P.append(day[1])
        df[asset+'_price'] = P
        df['timestamp']  = tt
    df['timestamp'] = tt
    for asset in list(names.keys()):#[:LIMIT]:
        
        diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(10)'] = change
        
        diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(1)'] = change        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)

    df = df[:-10]

    print("CHECK POINT ")
    return df

In [ ]:
names = {"DAI"  : 'dai',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDT"  : "tether",
        "ETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "WBTC2"  : "wrapped-bitcoin",
        "SUSHI" :"sushi",  
        "YFI"   : "yearn-finance",
        "AAVE"  : "aave",
        "BAT"   : "basic-attention-token",
        "ZRX"   : "0x",
        "REP"   : "augur",
        "SAI"   : "sai",
        }

def change():    
    columns = []
    columns.append('timestamp')
    print(names)
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        print("DEBUG 1 ", asset)
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data = request.json()
        tt = []
        P = []
        # print("DEBUG 1 ", data)
        if len(data['prices']) < LOOK_BACK:
            req = LOOK_BACK - len(data['prices'])
            addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
            data['prices'] = addition + data['prices']
            track[asset] = req
        else:
            data['prices'] = data['prices'][-LOOK_BACK:]

        for day in data['prices']:
            tt.append(day[0]/1000)
            P.append(day[1])
        df[asset+'_price'] = P
#         df['timestamp']  = tt
#     df['timestamp'] = tt
    for asset in list(names.keys()):#[:LIMIT]:

        diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(10)'] = change

        diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(1)'] = change        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)
    df = df[:-10]
    print("CHECK POINT ")
    return df[1:]

change_df = change()

# min_var = 9999999

In [ ]:
change_df.head()

In [ ]:
# !mkdir AAVE_COMPOUND_DATA
!mkdir COMPOUND_DATA

In [ ]:
# names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         # "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         # "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         # # "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         # "REN"   : "ren",
#         "RENFIL": "renfil",
#         # "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }

In [ ]:
# requiredDate

In [ ]:
# Compound Tokens
names = {"DAI"  : 'dai',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDT"  : "tether",
        "ETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "WBTC2"  : "wrapped-bitcoin",
        "SUSHI" :"sushi", # check -- - - - - -- 
        "YFI"   : "yearn-finance",
        "AAVE"  : "aave",
        "BAT"   : "basic-attention-token",
        "ZRX"   : "0x",
        "REP"   : "augur",
        "SAI"   : "sai",
        }

In [ ]:
ind  = 0 
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
df   = pd.DataFrame(columns = list(res.json()['cToken'][0].keys()))
for ctoken in res.json()['cToken'][3:]:
    
    print(ctoken['symbol'])
    if ctoken['symbol'] == 'cCOMP':
        continue

    ind += 1
    for key in ctoken.keys():
        try:
            df.at[ind,key ] = ctoken[key]['value']
        except:
            df.at[ind,key ] = ctoken[key]

    

    address   = ctoken['token_address']

    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[ctoken['symbol'][1:]])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    prices_df = pd.DataFrame(columns=['timestamp', 'price'])
    timestamp = []
    prices    = []

    for req in request.json()['prices']:
        timestamp.append(int(float(req[0])/1000))
        prices.append(req[1])

    prices_df['timestamp'] = timestamp
    prices_df['price']     = prices

    date= []
    for time in timestamp:
        date.append(datetime.fromtimestamp(time).strftime('%Y-%m-%d'))
    prices_df['date']  = date

In [ ]:
prices_df

In [ ]:
requiredDate
datetime.strptime(requiredDate, '%m-%d-%Y').strftime('%Y-%m-%d')

In [ ]:
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
list(res.json()['cToken'][0])

In [ ]:
ind  = 0 
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
df   = pd.DataFrame(columns = list(res.json()['cToken'][0].keys()))
# MASTER_DATE = '2021-12-20'
MASTER_DATE = datetime.strptime(requiredDate, '%m-%d-%Y').strftime('%Y-%m-%d')
for ctoken in res.json()['cToken']:
    try:
        ind += 1
        for key in ctoken.keys():
            try:
                df.at[ind,key ] = ctoken[key]['value']
            except:
                df.at[ind,key ] = ctoken[key]


        address   = ctoken['token_address']

        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[ctoken['symbol'][1:]])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        prices_df = pd.DataFrame(columns=['timestamp', 'price'])
        timestamp = []
        prices    = []

        for req in request.json()['prices']:
            timestamp.append(int(float(req[0])/1000))
            prices.append(req[1])

        prices_df['timestamp'] = timestamp
        prices_df['price']     = prices

        date= []
        for time in timestamp:
            date.append(datetime.fromtimestamp(time).strftime('%Y-%m-%d'))
        prices_df['date']  = date

#         datestamp = '2021-12-14'
        datestamp = MASTER_DATE
        max_timstamp = int(datetime.strptime(datestamp, '%Y-%m-%d').timestamp()) + 86400
        SUPPLY_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
        BORROW_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
        DF = pd.DataFrame()
        BIND = 0
        SIND = 0
        for _ in range(5):
            min_timestamp = max_timstamp  -  30 * 86400 
            res = requests.get("https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=%s&max_block_timestamp=%s&num_buckets=50" % ( address,min_timestamp, max_timstamp));    
            max_timstamp = max_timstamp   - 30 * 86400 
            temp = res.json()


            for tem in temp['total_borrows_history']:
                BORROW_DF.loc[BIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]                     
                BIND += 1


            for tem in temp['total_supply_history']:
                SUPPLY_DF.loc[SIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]
                SIND += 1                      


            print("***")

        BORROW_DF = BORROW_DF.sort_values(['block_timestamp'])
        BORROW_DF['DATE'] = BORROW_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
        BORROW_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)
        SUPPLY_DF = SUPPLY_DF.sort_values(['block_timestamp'])
        SUPPLY_DF['DATE'] = SUPPLY_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
        SUPPLY_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)

        SUPPLY_DF['supply_tokens'] = pd.to_numeric(SUPPLY_DF['value']) *  float(temp['exchange_rates'][-1]['rate'])


        BORROW_DF = BORROW_DF[::-1]
        SUPPLY_DF = SUPPLY_DF[::-1]
        DF['timestamp']    = BORROW_DF['block_timestamp']
        DF['BORROW']       = BORROW_DF['value']
        DF['SUPPLY']       = SUPPLY_DF['supply_tokens']
        DF['DATE']         = BORROW_DF['DATE']
        DF['SUPPLY_VALUE'] = SUPPLY_DF['value']


        assigned_price = []
        for indW in DF['DATE'] :
            a = prices_df[prices_df['date']==str(indW)]
            assigned_price.append(a['price'].values[0])

        DF['prices'] = assigned_price

        DF['total_supply'] =  DF['SUPPLY'] * DF['prices']
        DF['total_borrow'] =  pd.to_numeric(DF['BORROW']) * pd.to_numeric(DF['prices'])
        DF['POSITION']     =  DF['total_borrow'] - DF['total_supply']
        DF['POSITION($)']  =  DF['POSITION'] / pow(10,9)
        DF.to_csv('COMPOUND_DATA/DF_'+ctoken['symbol'][1:] + '.csv')



        print("**********************", DF.shape)
    except Exception as e:
        print("ERROR ", e, ctoken['symbol'])
    

In [ ]:
fils = os.listdir('COMPOUND_DATA/')

In [ ]:
cols = []
for fil in fils:
    try:
        aa = fil.split('.')[0].split('_')
    #     print(aa[1])
        cols.append(aa[1])
    except:
        pass

In [ ]:
FINAL ={}
for key in cols:
    try:
        df = pd.read_csv('COMPOUND_DATA/'+'DF_'+key+'.csv')
        df = df.sort_values(['timestamp'])
#         print(df.head())
        df = df[::-1]
        FINAL[key] = {'Assets': df['POSITION($)'],
                      'date':df['DATE'],
                      'MARKET':df['total_supply'],
                      'BORROW':df['total_borrow']
                     }
    
    except Exception as e:
        print(e, key)


In [ ]:
ff = []
for f in FINAL['ETH']['date']:
    ff.append(f)

In [ ]:
FINAL["AAVE"]["date"]

In [ ]:
FINAL_DF = pd.DataFrame(columns= cols)
ind = 0
for i in range(100):
    assets = []
    for col in cols:
        try:
            assets.append(FINAL[col]['Assets'][i])
        except:
            assets.append([])
    
    FINAL_DF.loc[ind] = assets
    ind += 1

In [ ]:
FINAL_DF['DATE'] = ff[:100]
FINAL_DF['timestamp'] = FINAL_DF['DATE'].apply(lambda x: pd.Timestamp(x).timestamp())
FINAL_DF = FINAL_DF.sort_values('timestamp')
FINAL_DF = FINAL_DF[::-1]

In [ ]:
# Assets for Compound Datewise
FINAL_DF

In [ ]:
change_df

In [ ]:
VAR = []
VAR_DATE = []
for ind, ROW in enumerate(FINAL_DF.iterrows()):
    REQUIRED_10 = []
    for required_row in change_df[ind:ind+365].iterrows():
        SUM_10 = []
        for key in ROW[1].keys()[:-3]:
            if key == 'WBTC2':
                key = 'WBTC'
            if isinstance(ROW[1][key], list):
                ROW[1][key] = 0
            SUM_10.append((float(ROW[1][key]) * required_row[1][key+'_%change(10)'])/100)
        REQUIRED_10.append(sum(SUM_10))
    VAR.append(sorted(REQUIRED_10, key=float)[3])
    VAR_DATE.append(change_df[ind:ind+365].at[REQUIRED_10.index(sorted(REQUIRED_10, key=float)[3]) + ind, 'date'])

In [ ]:
BORROW_DF

In [ ]:
# VAR_DATE

In [ ]:
FINAL_DF["DATE"]

In [ ]:
# Sample Final Res Columns
resultDf.columns

In [ ]:
# resultDf

In [ ]:
FINAL_DF.columns[:-2][0]

In [ ]:
requiredDate

In [ ]:
FINAL_DF["AAVE"].loc[0]

In [ ]:
VAR_DATE[0]

In [ ]:
idx = change_df.index[change_df['date'] == str(VAR_DATE[0])]
priceChangeVARDATE = change_df.loc[idx]

In [ ]:
priceChangeVARDATE

In [ ]:
idx = change_df.index[change_df['date'] == str(VAR_DATE[0])]
priceChangeVARDATE = change_df.loc[idx]
    
for token in list(FINAL_DF.columns[:-2]):
    try:
#         print(token)
        temp = []
        
        temp.append("COMPOUND")
        temp.append(str(token))
        temp.append(float(FINAL_DF[token].loc[0]*pow(10,9)))
        temp.append(float(FINAL_DF[token].loc[0]))
        temp.append(priceChangeVARDATE[token+"_%change(10)"].iloc[0])
        temp.append(float(-1 * float(FINAL_DF[token].loc[0]) * float(priceChangeVARDATE[token+"_%change(10)"].iloc[0])))
        
        # Add the values to the latest row
        df_length = len(resultDf)
        resultDf.loc[df_length] = temp
        
    except:
        print("Exception occured for the token: "+str(token))
# print(temp)

In [ ]:
resultDf

In [ ]:
resultDf.to_csv(str(requiredDate)+"_AAVE_COMPOUND_VAR.csv", index=False)

In [ ]:

# # for token in aaveV2Res["Token"]:
# for idx, row in aaveV2Res.iterrows():
#     try:
#         temp = []
        
#         temp.append("AAVE")
#         temp.append(str(aaveV2Res["Token"][idx]))
#         temp.append(aaveV2Res["Position"][idx])
#         temp.append(aaveV2Res["Position($,Billion)"][idx])
#         temp.append(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])
#         temp.append(float(-1 * float(aaveV2Res["Position($,Billion)"][idx]) * float(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])))
        
        
        
#         # Add the values to the latest row
#         df_length = len(resultDf)
#         resultDf.loc[df_length] = temp
        
#     except:
#         print("Exception occured for the token: "+str(token))

# # print(temp)

In [ ]:
# idx = FINAL_DF.index[FINAL_DF['DATE'] == MASTER_DATE].tolist
# print(idx)
# # COMPpriceChangeVARDATE = FINAL_DF.loc[idx]

In [ ]:
# FINAL_DF.index[FINAL_DF['DATE'] == MASTER_DATE]

In [ ]:
# FINAL_DF["DATE"]

In [ ]:
# MASTER_DATE